In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import os

In [2]:
script_directory = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
relative_folder_path = "data"
folder_path = os.path.join(script_directory.replace('\code', ''), relative_folder_path)
file = os.listdir(folder_path)
path = f"{folder_path}\{file[0]}"
df_root = pd.read_excel(path)

In [3]:
from datetime import datetime

print(float((datetime.strptime('11:00', '%H:%M') - datetime.strptime('10:00', '%H:%M')).total_seconds() / 60))

60.0


In [3]:
for i in df_root.Campo.unique():
    if 'rand' in i.lower():
        print(i)

Is the participant eligible to randomization?
Will the participant be randomized?
Is the subject replacing a subject previously been randomized in the current study?


In [16]:
df_root.Visit.unique() 

array(['Screening Visit', 'D-1', 'D1', 'D2', 'D3', 'D4', 'D7', 'D14',
       'D15', 'D16', 'D17', 'D18', 'D21', 'D28', 'D29', 'D30', 'D31',
       'D32', 'D35', 'D42', 'D63', '-'], dtype=object)

In [27]:
df_root.columns

Index(['Id Form', 'idCampo', 'Site', 'Visit', 'name', 'Genero',
       'AñoNacimiento', 'NameActividad', 'activityState', 'fecha enrolado',
       'Participante', 'Estado del Participante', 'identifierPrefix',
       'Id Activityins', 'Instancia', 'Estado del campo', 'Campo', 'Valor',
       'displayName', 'id option', 'Variable', 'visible', 'required',
       'TipoDato', 'TypeQuery', 'MessageQuery', 'QueryState',
       'vistaPrincipal.IdSubject'],
      dtype='object')

In [4]:
df_root.name.unique()

array(['Date of visit', 'Informed Consent', 'Demographics',
       'History of cutaneous leishmaniasis', 'Covid 19 testing',
       'Eligibility',
       'Medical Or Surgical History (other than Leishmaniasis)',
       'Vein assessment', 'Urinary Drug Screen', 'Alcohol Screen',
       'Clinical Laboratory - Test Hematology',
       'Clinical Laboratory Test - Clinical Chemistry',
       'Clinical Laboratory Test - Coagulation',
       'Immunoassay (Thyroid Stimulating Hormone)', 'Urinalysis',
       'Urine Microscopic Examination', 'Virology', 'Lesion Measurement',
       'Physical Examination', 'Vital Signs', '12-Lead ECG',
       'Clinical Laboratory Test - Clinical Chemistry D-1',
       'Interleukin-6', 'Titration Of Auto-Antibodies',
       'Injection Site Examination', 'PBMC Isolate',
       'Pharmacokinetic Blood Sampling (PK)',
       'Pharmacodynamic Blood Sampling (PD) -Cytokines/Chemokines',
       'mRNA Markers', 'CpG ODN D35 Administration',
       'End of Study Treatment 

In [5]:
df_root.Visit.unique()  

array(['Screening Visit', 'D-1', 'D1', 'D2', 'D3', 'D4', 'D7', 'D14',
       'D15', 'D16', 'D17', 'D18', 'D21', 'D28', 'D29', 'D30', 'D31',
       'D32', 'D35', 'D42', 'D63', '-'], dtype=object)

In [7]:
formulario_elegido =  'Clinical Laboratory Test - Clinical Chemistry'
visita_elegida = 0
campo_elegido = 0

general_revision  = df_root[df_root['name']== formulario_elegido]
general_revision = general_revision[['Visit','Participante', 'Campo', 'Valor']]
if visita_elegida != 0:
    general_revision = general_revision[general_revision['Visit'] == visita_elegida]

if campo_elegido != 0:
    general_revision = general_revision[general_revision['Campo']== campo_elegido]

general_revision = general_revision[['Visit', 'Campo', 'Participante','Valor']]

In [8]:
general_revision.Campo.unique()

array(['Blood Sample Collected', 'Date Collected', 'Time Collected',
       'Fasting', 'Alkaline Phosphatase, Result (U/L)',
       'Alkaline Phosphatase, Out of normal range?',
       'Gamma Glutamyl Transferase (GGT), Result (U/L)',
       'Gamma Glutamyl Transferase (GGT), Out of normal range?',
       'Bilirubin (Total), Result (mg/dL)',
       'Bilirubin (Total), Out of normal range?',
       'Bilirubin (Direct) (only if Total is elevated), Result (mg/dL)',
       'Bilirubin (Direct) (only if Total is elevated), Out of normal range?',
       'Albumin, Result (g/dL)', 'Albumin, Out of normal range?',
       'Albumin, If abnormal, Specify',
       'Aspartate Aminotransferase (AST), Result (U/L)',
       'Aspartate Aminotransferase (AST), Out of normal range?',
       'Alanine Aminotransferase (ALT), Result (U/L)',
       'Alanine Aminotransferase (ALT), Out of normal range?',
       'Protein (Total), Result (mg/dL)',
       'Protein (Total), Out of normal range?',
       'Creatine K

In [9]:
general_revision[general_revision['Participante']== '011001' ].head(50)

,Visit,Campo,Participante,Valor
479,D1,Was any pharmacokinetic blood sample collected?,011001,1
480,D1,Date of blood sample collected,011001,31-OCT-2023
481,D1,"Pre dose, Time",011001,08:40
482,D1,"05-min post dose, Time",011001,09:41
483,D1,"10-min post dose, Time",011001,09:46
484,D1,"15-min post dose, Time",011001,09:51
485,D1,"20-min post dose, Time",011001,09:56
486,D1,"25-min post dose, Time",011001,10:01
487,D1,"30-min post dose, Time",011001,10:06
488,D1,"45-min post dose, Time",011001,10:24


In [28]:

df_time_dosing12 = df_root[df_root['name']=='CpG ODN D35 Administration'].sort_values(by='Instancia')
df_time_dosing12 = df_time_dosing12[(df_time_dosing12['Campo']=='Date of dosing') | (df_time_dosing12['Campo']=='Time of Dosing')]
df_time_dosing2 = df_time_dosing12[df_time_dosing12['Campo']=='Date of dosing']
df_time_dosing2['time_dosing_cpg_administration'] =  df_time_dosing12[df_time_dosing12['Instancia'].isin(df_time_dosing2['Instancia'] + 1) & (df_time_dosing12['Campo'] == 'Time of Dosing')]['Valor'].values
df_time_dosing2 =df_time_dosing2[['Participante','Valor', 'time_dosing_cpg_administration']]
df_time_dosing2 = df_time_dosing2.rename(columns={'Participante':'Subject', 'Valor':'date_ex_to_join2', 'time_dosing_cpg_administration': 'time_dosing_cpg_administration2'})

C:\Users\sebastian sossa\AppData\Local\Temp\ipykernel_10992\1203940693.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time_dosing2['time_dosing_cpg_administration'] =  df_time_dosing12[df_time_dosing12['Instancia'].isin(df_time_dosing2['Instancia'] + 1) & (df_time_dosing12['Campo'] == 'Time of Dosing')]['Valor'].values


In [30]:
general_revision[general_revision['Campo']== 'Date of dosing' ].head(50)

,Visit,Campo,Participante,Valor
2603,-,Date of dosing,011001,31-OCT-2023
2888,-,Date of dosing,011001,14-NOV-2023
2894,-,Date of dosing,011001,28-NOV-2023


In [ ]:
formulario_elegido =  'CpG ODN D35 Administration'
visita_elegida = 0
campo_elegido = 0

general_revision  = df_root[df_root['name']== formulario_elegido]
general_revision = general_revision[['Visit','Participante', 'Campo', 'Valor']]
if visita_elegida != 0:
    general_revision = general_revision[general_revision['Visit']in ['D2', 'D16', 'D30']]

if campo_elegido != 0:
    general_revision = general_revision[general_revision['Campo']== campo_elegido]

general_revision = general_revision[['Visit', 'Campo', 'Participante','Valor']]

In [29]:
df_time_dosing2

,Subject,date_ex_to_join2,time_dosing_cpg_administration2
2603,011001,31-OCT-2023,09:36
2888,011001,14-NOV-2023,09:05
2894,011001,28-NOV-2023,10:12


In [8]:
general_revision[general_revision['Campo']=='60-mins post dose, Time']

,Visit,Campo,Participante,Valor


In [9]:
df__lession  = df_root[df_root['name']=='Lesion Measurement']
df__lession = df__lession[['Visit','Participante', 'Campo', 'Valor']]
df__lession = df__lession[df__lession['Visit']=='Screening Visit']
df__lession = df__lession[df__lession['Campo']=='Anatomical Location']
df__lession = df__lession[['Visit','Participante','Valor']]
df__lession = df__lession.rename(columns={'Participante':'Subject', 'Valor':'lesion_measurement'})
df__lession['Count'] = len(df__lession['lesion_measurement'])
df__lession = df__lession[['Visit','Subject','Count']]
# df_with_counts = df__lession.merge(subject_counts, left_on='Subject', right_on='Subject')
# df_with_counts = df_with_counts[['Visit','Subject','Count']]

In [10]:
df_root[df_root['Campo']=='Date of ECG Performed'].Valor.unique()

array(['25-OCT-2023', '31-OCT-2023', '14-NOV-2023', '28-NOV-2023',
       '12-DEC-2023'], dtype=object)

In [11]:
prueba

NameError: name 'prueba' is not defined

In [ ]:
df__lession

,Visit,Subject,Count
300,Screening Visit,011001,2
309,Screening Visit,011001,2


In [ ]:
df_root.activityState.unique()

array(['DATA_VERIFIED', 'DATA_ENTRY_COMPLETE', 'DATA_ENTRY_INCOMPLETE',
       'DATA_EMPTY'], dtype=object)

In [ ]:
df_root.Participante.unique()

array(['011001', 'S002', 'S003'], dtype=object)

In [ ]:
prueba = df_root[df_root['name']=='CpG ODN D35 Administration'].sort_values(by='Instancia')
prueba = prueba[(prueba['Campo']=='Date of dosing') | (prueba['Campo']=='Time of Dosing')]
prueba1 = prueba[prueba['Campo']=='Date of dosing']
prueba1['time_dosing_cpg_administration'] =  prueba[prueba['Instancia'].isin(prueba1['Instancia'] + 1) & (prueba['Campo'] == 'Time of Dosing')]['Valor'].values
prueba1 =prueba1[['Participante', 'Campo', 'Valor', 'time_dosing_cpg_administration' ,'Instancia']]
prueba1 = prueba1.rename(columns={'Participante':'Subject', 'Valor':'date_cpg_administration'})
prueba1


C:\Users\sebastian sossa\AppData\Local\Temp\ipykernel_21716\1665216494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba1['time_dosing_cpg_administration'] =  prueba[prueba['Instancia'].isin(prueba1['Instancia'] + 1) & (prueba['Campo'] == 'Time of Dosing')]['Valor'].values


,Subject,Campo,date_cpg_administration,time_dosing_cpg_administration,Instancia
2603,011001,Date of dosing,31-OCT-2023,09:36,17797
2888,011001,Date of dosing,14-NOV-2023,09:05,21664
2894,011001,Date of dosing,28-NOV-2023,10:12,21678


In [ ]:
df_time_dosing1 = df_root[df_root['name']=='CpG ODN D35 Administration'].sort_values(by='Instancia')
df_time_dosing1 = df_time_dosing1[(df_time_dosing1['Campo']=='Date of dosing') | (df_time_dosing1['Campo']=='Time of Dosing')]
df_time_dosing = df_time_dosing1[df_time_dosing1['Campo']=='Date of dosing']
df_time_dosing['time_dosing_cpg_administration'] =  df_time_dosing1[df_time_dosing1['Instancia'].isin(df_time_dosing['Instancia'] + 1) & (df_time_dosing1['Campo'] == 'Time of Dosing')]['Valor'].values
df_time_dosing =df_time_dosing[['Participante', 'Campo', 'Valor', 'time_dosing_cpg_administration']]
df_time_dosing = df_time_dosing.rename(columns={'Participante':'Subject', 'Valor':'date_ex_to_join'})

C:\Users\sebastian sossa\AppData\Local\Temp\ipykernel_21716\1862077006.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time_dosing['time_dosing_cpg_administration'] =  df_time_dosing1[df_time_dosing1['Instancia'].isin(df_time_dosing['Instancia'] + 1) & (df_time_dosing1['Campo'] == 'Time of Dosing')]['Valor'].values


In [ ]:
df_time_dosing

,Subject,Campo,date_ex_to_join,time_dosing_cpg_administration
2603,011001,Date of dosing,31-OCT-2023,09:36
2888,011001,Date of dosing,14-NOV-2023,09:05
2894,011001,Date of dosing,28-NOV-2023,10:12


In [ ]:
df_time_dosing = df_root[df_root['name']=='CpG ODN D35 Administration']
df_time_dosing = df_time_dosing[['Participante', 'Campo', 'Valor', 'FormFieldInstance Id']]
df_time_dosing = df_time_dosing[df_time_dosing['Campo']=='Was the visit performed?']
df_time_dosing = df_time_dosing[['Visit','Participante','Valor_completo']]
df_time_dosing = df_time_dosing.rename(columns={'Participante':'Subject', 'Valor_completo':'was_DV_performed'})

KeyError: "['FormFieldInstance Id'] not in index"

In [ ]:
df_contraception = df_root[df_root['name']== 'Prior And Concomitant Medications']
df_contraception = df_contraception[['Visit','Participante', 'Campo', 'Valor', 'Instancia']]
df_contraception = df_contraception.sort_values(by=['FormFieldInstance Id'])
df_contraception = df_contraception.reset_index(drop=True)
date_indices = df_contraception.index[df_contraception['Campo'] == 'Concomitant Medication ID'].tolist()
subdatasets = [df_contraception.iloc[start:end] for start, end in zip(date_indices, date_indices[1:] + [None])]
df_to_join = pd.DataFrame()
for sub in subdatasets:
    #if sub['Valor'].tolist()  in [5, 5.0, '5', '5.0', 9]:
    if len([float(x) for x in sub['Valor'].tolist() if x in [5, 5.0, '5', '5.0', 9]]) != 0:
        df_to_join = sub[(sub['Campo'] == 'Indication Category') | (sub['Campo'] == 'Start date')]
        sujeto = df_to_join.Participante.unique()
        df_to_join = df_to_join[['Campo', 'Valor']].T
        new_columns = df_to_join.iloc[0]
        df_to_join = df_to_join[1:].set_axis(new_columns, axis=1)
        df_to_join['Subject'] = sujeto
        break
df_to_join = df_to_join.rename(columns={'Participante':'Subject', 'Start date':'start_date_combined_hormonal'})

KeyError: 'FormFieldInstance Id'

In [ ]:
df_to_join

Campo,Birth Month,Birth Year,Subject
Valor,5,2000,S003


In [ ]:
pru.Campo.unique()

array(['Was the ECG performed?', 'Date of ECG Performed',
       'Undefined, HR (bpm)', 'Undefined, RR (msec)',
       'Undefined, PR (msec)', 'Undefined, QRS (msec)',
       'Undefined, QT (msec)', 'Undefined, QTcF (msec)',
       'Undefined, Interpretation', 'Pre dose triplicate 1, Time 24 hrs',
       'Pre dose triplicate 1, HR (bpm)',
       'Pre dose triplicate 1, RR (msec)',
       'Pre dose triplicate 1, PR (msec)',
       'Pre dose triplicate 1, QRS (msec)',
       'Pre dose triplicate 1, QT (msec)',
       'Pre dose triplicate 1, QTcF (msec)',
       'Pre dose triplicate 1, Interpretation',
       'Pre dose triplicate 1, If abnormal, specify',
       'Pre dose triplicate 1, Please specify abnormal interpretation',
       'Pre dose triplicate 2, Time 24 hrs',
       'Pre dose triplicate 2, HR (bpm)',
       'Pre dose triplicate 2, RR (msec)',
       'Pre dose triplicate 2, PR (msec)',
       'Pre dose triplicate 2, QRS (msec)',
       'Pre dose triplicate 2, QT (msec)',
       

In [ ]:
triplicate_1 = pru[pru['Campo']=='Pre dose triplicate 1, Time 24 hrs'].Valor.unique()[0]
triplicate_2 = pru[pru['Campo']=='Pre dose triplicate 2, Time 24 hrs'].Valor.unique()[0]
print(triplicate_1)
print(triplicate_2)

08:20
08:25


In [ ]:
from datetime import datetime
time_format = '%H:%M'
triplicate_1_formated = datetime.strptime(triplicate_1, time_format)
triplicate_2_formated = datetime.strptime(triplicate_2, time_format)

time_difference = (triplicate_2_formated - triplicate_1_formated).total_seconds() / 60
print(time_difference)


5.0


In [ ]:
import math
import numpy as np
prueba_variable = np.nan

if math.isnan(float(prueba_variable)):
    print('Si funciona asi')
else:
    print('No funciona asi') 

Si funciona asi


In [ ]:
import math

if math.nan > 4:
    print('Si toca cambiar')
else:
    print('no toca cambiar')

no toca cambiar
